In [1]:
import numpy as np
import pandas as pd

In [19]:
train_df = pd.read_csv('resources/train.csv', index_col=0)
test_df = pd.read_csv('resources/test.csv', index_col=0)

In [22]:
train_df.head()

,delta,start.1,start.2,start.3,start.4,start.5,start.6,start.7,start.8,start.9,...,stop.391,stop.392,stop.393,stop.394,stop.395,stop.396,stop.397,stop.398,stop.399,stop.400
id,,,,,,,,,,,,,,,,,,,,,
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,2,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0


In [23]:
row = train_df.head(1)

In [65]:
def generate_windows(field, delta=1):
    padded = np.pad(field, delta, 'constant', constant_values=-1)
    rows = []
    
    n = len(field)
    for i in range(n):
        for j in range(n):
            window = padded[i:i+2*delta+1, j:j+2*delta+1]
            rows.append(window.ravel())
    return np.array(rows)

In [73]:
from tools import extract_features_from_raw_data

In [100]:
fst_delta = train_df[train_df.delta == 1].values
X, y = fst_delta[:,1:401], fst_delta[:,401:]

In [101]:
import sklearn

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [114]:
X_train.shape

(7410, 400)

In [105]:
def train_sample_windowize(field, delta=1, n=20):
    """ Same as the above, but with custom delta """
    padded = np.pad(field, delta, mode='constant', constant_values=-1)
    X = np.zeros((n * n, (1 + delta * 2) ** 2))
    for i in range(n):
        for j in range(n):
            X[i * n + j] = padded[i:i + 2 * delta + 1, j:j + 2 * delta + 1].ravel()
    return X

In [124]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error

In [164]:
def window_data_proc(X, y=None, delta=1):
    """ Reformat data in window form and prepare for training """
    vectorize_windowing = lambda row: train_sample_windowize(row.reshape(20, 20), delta=delta)

    X = np.vstack(np.apply_along_axis(vectorize_windowing, 1, X))
    if y is not None:
        y = np.vstack(y.ravel())
        return X, y
    return X

In [145]:
X2_train, y2_train = window_data_proc(X_train, y_train, 2)
X2_test, y2_test = window_data_proc(X_test, y_test, 2)

In [146]:
lr = LogisticRegression()
lr.fit(X2_train, y2_train)

/Volumes/Storage/goinfre/ptyshevs/dl_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Volumes/Storage/goinfre/ptyshevs/dl_env/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [147]:
y_pred = lr.predict(X2_test)
mean_absolute_error(y2_test, y_pred)

0.1140587044534413

In [148]:
from sklearn.ensemble import RandomForestClassifier

In [150]:
rf = RandomForestClassifier(100, class_weight={0: 0.3, 1:0.7})
rf.fit(X2_train, y2_train)

/Volumes/Storage/goinfre/ptyshevs/dl_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight={0: 0.3, 1: 0.7},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [154]:
y_pred = rf.predict(X2_test)
mean_absolute_error(y2_test, y_pred)

1.5182186234817814e-05

In [155]:
rf2 = RandomForestClassifier(100)
rf2.fit(X2_train, y2_train)

/Volumes/Storage/goinfre/ptyshevs/dl_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [156]:
y_pred = rf2.predict(X2_test)

1.1133603238866398e-05

In [157]:
mean_absolute_error(y2_test, y_pred) * 100

0.0011133603238866398

In [158]:
trd_delta = train_df[train_df.delta == 3].values
X, y = trd_delta[:,1:401], trd_delta[:,401:]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [165]:
X2_train, y2_train = window_data_proc(X_train, y_train, 3)
X2_test, y2_test = window_data_proc(X_test, y_test, 3)

In [166]:
y2_test.shape

(996400, 1)

In [167]:
rf = RandomForestClassifier(100, class_weight={0: 0.3, 1:0.7})
rf.fit(X2_train, y2_train)
y_pred = rf.predict(X2_test)
mean_absolute_error(y2_test, y_pred)

/Volumes/Storage/goinfre/ptyshevs/dl_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.1013849859494179

In [172]:
n_features = X2_train.shape[1]
n_features

49

In [182]:
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout

nn = Sequential()
nn.add(Dense(400, input_shape=(n_features,), activation='relu'))
nn.add(Dense(400, activation='relu'))
nn.add(Dense(400, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))
nn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['mae'])

In [ ]:
nn.fit(X2_train, y2_train)

Epoch 1/1
 289472/2989200 [=>............................] - ETA: 5:09 - loss: 0.2719 - mean_absolute_error: 0.1726

In [ ]:
pred = nn.predict(X2_test)
mean_absolute_error